In [ ]:
import uproot as up
import matplotlib.pyplot as plt
import numpy as np
import awkward as ak
import re
import pandas as pd
from uproot3_methods.classes.TLorentzVector import TLorentzVector
from tqdm.notebook import tqdm
import pandas as pd
import plotly.express as ps
import plotly.graph_objects as go

In [ ]:
class rrFpy_pandas:
    def __init__(self, file_name=None):
        if type(file_name) == str:
            self.load_ROOT(file_name)
        else:
            self.df_pivoted = pd.DataFrame()
            self.filter = pd.Series(dtype = bool)
    def load_ROOT(self, file_name):
        file = up.open(file_name)
        ntp = file["ntp"]
        df = pd.DataFrame(index = ak.to_pandas(ntp["pz"].array()).index, columns=["px", "py", "pz", "E", "Id"])
        df["px"] = ak.to_pandas(ntp["px"].array())
        df["py"] = ak.to_pandas(ntp["py"].array())
        df["pz"] = ak.to_pandas(ntp["pz"].array())
        df["E"] = ak.to_pandas(ntp["E"].array())
        df["Id"] = ak.to_pandas(ntp["Id"].array())
        df = df.reset_index()
        self.df_pivoted = pd.pivot_table(df, index = "entry", columns="subentry", values = ["px","py", "pz","E", "Id"])
        self.df_pivoted["nTrk"] = ak.to_pandas(ntp["nTrk"].array())
        self.filter = pd.Series(index=self.df_pivoted.index, data=True)
        file.close()
        
    def sign(self, x):
        return -1 if x<0 else 1
    
    def parse_var(self, var):
        """
        parse_var(var): parses variables into name and list of particles
        """
        [name, numbers] = var.split("_")
        part_list = [int(n) for n in re.sub('([0-9])',r'\1 ',numbers.replace('m','-')).split(' ')[:-1]]
        return name, part_list
    
    def get_momDF_from_partList(self, part_list):
        """
        Constructs new dataFrame with 'px', 'py', 'pz', 'E' fields polulated with components of 
        total momentum that corresponds to part_list
        """
        df_res = pd.DataFrame(index=self.df_pivoted.index, columns = ["px","py","pz","E", "out"]).fillna(0)
        for i in part_list:
            abs_i = abs(i)
            for v in ["px","py","pz","E"]:
                df_res[v] += self.sign(i)*self.df_pivoted[v,abs_i]
        return df_res
        
    def __getitem__(self, key):
        """
        [var] operator: constructs the table of the variable for given dataset (taking cuts into account)
        """
        if type(key) == str:   # extract one var as a Series
            if key == "ntr":
                df_res = self.df_pivoted["nTrk"]
                df_res = df_res[ self.filter]
                return df_res
            name, part_list = self.parse_var(key)
            mom_functions_dict =  {'m2': lambda t, x, y, z: t**2-x**2-y**2-z**2,
                'm': lambda t, x, y, z: np.sqrt(t**2-x**2-y**2-z**2),
                'pt2': lambda t, x, y, z: x**2+y**2,
                 'pt': lambda t, x, y, z: np.sqrt(x**2+y**2)};
            if name in mom_functions_dict.keys():
                func =mom_functions_dict[name]
                df_res = self.get_momDF_from_partList(part_list)
                df_res = df_res[ self.filter]
                return func(df_res["E"], df_res["px"], df_res["py"], df_res["pz"])
            elif name == "id":
                df_res = self.df_pivoted["Id", abs(part_list[0])]
                df_res = df_res[ self.filter]
                return df_res
        elif type(key) == list:  # exctract many vars and DataFrame
            df_all =  pd.concat([self[k] for k in key], axis=1)
            df_all.columns = key
            return df_all
    
    def reset_cuts(self):
        """
        Resets all filters set by previous cuts
        """
        self.filter = pd.Series(index=self.df_pivoted.index, data=True)

    def cut(self, condition, in_place = False):
        """
        cut(condition): imposes a cut to the given dataset. Can be restored with reset_cuts
        """
        new_filter = self.filter.copy()
        if type(condition) == str:
            condition = re.sub('([<>=])',r' \1 ', condition)
            var, operation, lhs = [s for s in condition.split(' ') if len(s)>0]
            if(operation == '>'):
                lhs = float(lhs)
                new_filter = self.filter & (self[var] > lhs) 
            elif operation == '<':
                lhs = float(lhs)
                new_filter = self.filter & (self[var] < lhs) 
            elif operation == '=':
                if '+-' in lhs:
                    lhs, error = [float(n) for n in lhs.split('+-')]
                else:
                    lhs = float(lhs)
                    error = 1e-3
                new_filter = self.filter & \
                    (self[var] > lhs - error) & (self[var] < lhs + error)
            result = rrFpy_pandas(None)
            result.df_pivoted = self.df_pivoted
            result.filter = new_filter.copy()
        return result

In [ ]:
rrF = rrFpy_pandas()
rrF.load_ROOT("../build/evtOutput.root")

In [ ]:
rrf_3pi = rrF.cut("ntr=5").cut("id_2=211").cut("id_3=211")
variables = {'m_23':r'$\pi^+\pi^+$', 'm_24':r'$\pi^+\pi^-$', 'm_34':r'$\pi^+\pi^-$'}
fig = go.Figure()
for var, name in variables.items():
    fig.add_trace(go.Histogram(x=rrf_3pi[var], name = name, histnorm='probability density'))
fig.update_layout(barmode='overlay',
                  xaxis_title=r"$m_{\pi\pi}, \mathrm{GeV}$",
                yaxis_title="$d\mathrm{Br}/dm_{\pi\pi}, \mathrm{GeV}^{-1}$",
                 title = r"$B_c^+\to J/\psi \pi^+\pi^+\pi^-$")
fig.update_traces(opacity=0.5)
fig.show()

In [ ]:
rrf_Kpipi = rrF.cut("ntr=5").cut("id_2=321").cut("id_3=211")
variables = {'m_23':r'$K^+\pi^+$', 'm_24':r'$K^+\pi^-$', 'm_34':r'$\pi^+\pi^-$'}
fig = go.Figure()
for var, name in variables.items():
    fig.add_trace(go.Histogram(x=rrf_Kpipi[var], name = name, histnorm='probability density'))
fig.update_layout(barmode='overlay',
                  xaxis_title=r"$m_{\pi\pi}, \mathrm{GeV}$",
                yaxis_title="$d\mathrm{Br}/dm_{\pi\pi}, \mathrm{GeV}^{-1}$",
                 title = r"$B_c^+\to J/\psi K^+\pi^+\pi^-$")
fig.update_traces(opacity=0.5)
fig.show()

In [ ]:
rrf_KKpi = rrF.cut("ntr=5").cut("id_2=321").cut("id_3=-321")
variables = {'m_23':r'$K^+K^-$', 'm_24':r'$K^+\pi^+$', 'm_34':r'$K^-\pi^+$'}
fig = go.Figure()
for var, name in variables.items():
    fig.add_trace(go.Histogram(x=rrf_KKpi[var], name = name, histnorm='probability density'))
fig.update_layout(barmode='overlay',
                  xaxis_title=r"$m_{\pi\pi}, \mathrm{GeV}$",
                yaxis_title="$d\mathrm{Br}/dm_{\pi\pi}, \mathrm{GeV}^{-1}$",
                 title = r"$B_c^+\to J/\psi K^+K^-\pi^+$")
fig.update_traces(opacity=0.5)
fig.show()